In [2]:
#Import All Libraries required
import requests as req
import pandas as pd
import numpy as np
import json

In [37]:
#Collecting Source Data for mapping fields

#1.Metric Data
metrics=pd.DataFrame(list(req.get(url="https://api.vortexweather.tech/api/metrics").json()))

#2.Source Data
sources=req.get(url="https://api.vortexweather.tech/api/sources").json()
src_list=[]
for i in range(0,len(sources)):
    for row in sources[i]['fields']:
        row['src_Name']=sources[i]['name']
    src_list.extend(sources[i]['fields'])
src_df=pd.DataFrame(src_list)

#3.Merging Metric Data to Source Data
metrics.rename({'id':'metric_id'},axis=1,inplace=True)
src_df=src_df.merge(metrics,on='metric_id').rename({'id':'src_field_id','name':'metric'},axis=1)

In [4]:
#Read all cities data to Dataframe
cities=pd.read_csv("https://raw.githubusercontent.com/kallsyms/wx_explore/master/data/cities/worldcities.csv",nrows=20)
cities=cities.iloc[:,:5]
cities.head()

,city,city_ascii,lat,lng,country
0,Tokyo,Tokyo,35.6850,139.7514,Japan
1,New York,New York,40.6943,-73.9249,United States
2,Mexico City,Mexico City,19.4424,-99.1310,Mexico
3,Mumbai,Mumbai,19.0170,72.8570,India
4,São Paulo,Sao Paulo,-23.5587,-46.6250,Brazil


In [5]:
#Function to extract city data
#Params-->city_data is dict/series of {city:<>,lat:<>,lng:<>,country:<>} format
#Returns-->city weather data in list format with city & country included
def get_citydata(city_data):
    country=city_data.country
    city=city_data.city
    lat=str(city_data.lat)
    lng=str(city_data.lng)
    payload={'lat':lat,'lon':lng}
    response=req.get(url="https://api.vortexweather.tech/api/wx",params=payload)
    j=response.json()['data']
    #Adding City & Country to weather data
    for key in j.keys():
        for item in j[key]:
            item['Ftime']=key
            item['city']=city
            item['country']=country
    print(city,":Data extracted")
    #Converting to list for ease of converting to DataFrame
    l=[]
    for key in j.keys():
        l.extend(list(j[key]))
    #Return Weather Data for city as list
    return l

In [27]:
#extracting weather data for top 20 cities
data_list=[]
for i in cities.index:
    data_list.extend(get_citydata(cities.iloc[i,:]))

Tokyo :Data extracted
New York :Data extracted
Mexico City :Data extracted
Mumbai :Data extracted
São Paulo :Data extracted
Delhi :Data extracted
Shanghai :Data extracted
Kolkata :Data extracted
Los Angeles :Data extracted
Dhaka :Data extracted
Buenos Aires :Data extracted
Karachi :Data extracted
Cairo :Data extracted
Rio de Janeiro :Data extracted
Ōsaka :Data extracted
Beijing :Data extracted
Manila :Data extracted
Moscow :Data extracted
Istanbul :Data extracted
Paris :Data extracted


In [30]:
#converting to dataframe
wdata=pd.DataFrame(data_list)
wdata.head()

,Ftime,city,country,raw_values,run_time,src_field_id,value
0,1594011600,Tokyo,Japan,[301.0],1593972000,121,301.000000
1,1594011600,Tokyo,Japan,[24134.779296875],1593972000,122,24134.779297
2,1594011600,Tokyo,Japan,[0.0],1593972000,123,0.000000
3,1594011600,Tokyo,Japan,[0.0],1593972000,124,0.000000
4,1594011600,Tokyo,Japan,[0.0],1593972000,125,0.000000


In [108]:
#Mapping wdata with source data frame (to map metrics)
wdata=wdata.merge(src_df,on='src_field_id')

In [150]:
#Exploring the Dataframe
wdata.head()

,Ftime,city,country,raw_values,run_time,src_field_id,value,metric_id,source_id,src_Name,metric,units
0,1594011600,Tokyo,Japan,[301.0],1593972000,121,301.000000,1,3,Global Forecast System,2m Temperature,K
1,1594011600,Tokyo,Japan,[298.9414978027344],1593777600,121,298.941498,1,3,Global Forecast System,2m Temperature,K
2,1594011600,Tokyo,Japan,[299.3316955566406],1593993600,121,299.331696,1,3,Global Forecast System,2m Temperature,K
3,1594011600,Tokyo,Japan,[300.9125671386719],1593950400,121,300.912567,1,3,Global Forecast System,2m Temperature,K
4,1594011600,Tokyo,Japan,[298.4999694824219],1593842400,121,298.499969,1,3,Global Forecast System,2m Temperature,K


In [156]:
#Extracting Temp Data
wdata[(wdata.city=='New York') & (wdata.metric_id==1)].groupby(['source_id','Ftime','run_time']).mean()

src_field_id       value  metric_id
source_id Ftime      run_time                                       
1         1594009800 1593946800            91  296.986176          1
                     1593950400            91  296.765045          1
                     1593954000            91  297.198090          1
                     1593957600            91  297.542328          1
                     1593961200            91  297.364777          1
                     1593964800            91  297.127411          1
                     1593968400            91  297.118988          1
                     1593972000            91  297.377838          1
                     1593975600            91  297.458801          1
                     1593979200            91  297.551788          1
                     1593982800            91  297.187866          1
                     1593986400            91  296.922974          1
                     1593990000            91  296.710052          1
                     1593993600            91  297.124817          1
                     1593997200            91  297.180878          1
                     1594000800            91  297.767090          1
                     1594004400            91  297.842010          1
                     1594008000            91  298.094299          1
          1594010700 1593946800            91  296.988373          1
                     1593950400            91  296.759277          1
                     1593954000            91  297.141205          1
                     1593957600            91  297.479156          1
                     1593961200            91  297.212982          1
                     1593964800            91  296.977875          1
                     1593968400            91  296.989227          1
                     1593972000            91  297.228943          1
                     1593975600            91  297.288116          1
                     1593979200            91  297.395050          1
                     1593982800            91  296.980011          1
                     1593986400            91  296.850281          1
...                                       ...         ...        ...
3         1594051200 1593756000           121  304.800018          1
                     1593777600           121  303.601288          1
                     1593799200           121  304.885315          1
                     1593820800           121  302.000000          1
                     1593842400           121  305.399994          1
                     1593864000           121  305.800018          1
                     1593885600           121  306.600006          1
                     1593907200           121  306.000000          1
                     1593928800           121  306.899994          1
                     1593950400           121  307.100006          1
                     1593972000           121  307.028076          1
                     1593993600           121  305.857117          1
          1594054800 1593626400           121  302.300018          1
                     1593648000           121  305.200012          1
                     1593669600           121  305.809082          1
                     1593691200           121  305.774506          1
                     1593712800           121  306.990204          1
                     1593734400           121  307.203949          1
                     1593756000           121  304.899994          1
                     1593777600           121  304.612122          1
                     1593799200           121  306.000000          1
                     1593820800           121  303.199982          1
                     1593842400           121  306.598907          1
                     1593864000           121  307.200012          1
                     1593885600           121  307.500000          1
                     1593907200

In [ ]:
#Visualiztion to be added